In [1]:
!pip install langchain openai langchain_openai

     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ---------------------------------------- 42.9/42.9 kB 2.0 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------- -------------------------------- 0.2/1.0 MB 4.0 MB/s eta 0:00:01
   ----------------------- ---------------- 0.6/1.0 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/55.0 kB ? eta -:--:--
   ---------------------------------------- 55.0/55.0 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/464.8 kB ? eta -:--:--
   --------------------------------------  460.8/464.8 kB 14.1 MB/s eta 0:00:01
   ---------------------------------------- 464.8/464.8 kB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/413.2 kB ? eta -:--:--
   -----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.1 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.1 which is incompatible.


In [2]:
!pip install langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB 1.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/2.5 MB 2.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.5 MB 4.2 MB/s eta 0:00:01
   ----------------- ---------------------- 1.1/2.5 MB 6.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.7/2.5 MB 8.1 MB/s eta 0:00:01
   ---------------------------------- ----- 2.2/2.5 MB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 8.9 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
   ---------------------------------------- 50.9/50.9 kB 2.7 MB/s eta 0:00:00
Using cached typing_inspect-0.9.0-py3-none-any.

In [20]:
!pip install streamlit

   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
   -- ------------------------------------ 30.7/413.4 kB 660.6 kB/s eta 0:00:01
   ---------- ----------------------------- 112.6/413.4 kB 1.1 MB/s eta 0:00:01
   ------------------------------- -------- 327.7/413.4 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 413.4/413.4 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.1
    Uninstalling protobuf-5.29.1:
      Successfully uninstalled protobuf-5.29.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.68.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "e0c632dca6c048298ac6d45161c11c0e"

In [4]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

In [18]:
import os
import time
from langchain_openai import AzureChatOpenAI
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage

In [19]:
# ✅ 1️⃣ LLM Tanımla
llm = AzureChatOpenAI(
    azure_endpoint="https://vrp-open-ai-test-02.openai.azure.com/",
    azure_deployment="veripark-chatgpt",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_version="2024-08-01-preview",
)

# ✅ 2️⃣ Tool'ları Tanımla
@tool
def money_transfer(recipient: str, amount: float) -> str:
    """Transfers money to a given recipient."""
    print(f"Processing money transfer to {recipient} for {amount} TL...")
    time.sleep(2)
    return f"Money transfer of {amount} TL to {recipient} completed successfully."

@tool
def address_update(new_address: str) -> str:
    """Updates the user's address."""
    print(f"Updating address to: {new_address}...")
    time.sleep(2)
    return f"Address update to {new_address} completed successfully."

@tool
def credit_card_payment(amount: float) -> str:
    """Pays the credit card bill."""
    print(f"Processing credit card payment of {amount} TL...")
    time.sleep(2)
    return f"Credit card payment of {amount} TL completed successfully."

tools = [money_transfer, address_update, credit_card_payment]

# ✅ 3️⃣ Memory Kullanımı (Konuşmayı hatırlaması için)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ 4️⃣ Agent Tanımla
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,
    verbose=True
)

# ✅ 5️⃣ Intent Detection (Niyet Belirleme)
def detect_intent(user_input):
    """
    Determines the intent of the user's message.
    Possible intents:
    - "money_transfer"
    - "address_update"
    - "credit_card_payment"
    - "greeting" (e.g., Hello, Hi)
    - "unknown" (if intent is unclear)
    """
    intent_prompt = f"""
    You are a helpful banking assistant. Your name is FinBotX
    Determine the intent of the following user message.

    Possible intents:
    - "money_transfer"
    - "address_update"
    - "credit_card_payment"
    - "greeting"
    - "unknown" (if the intent is unclear)

    User message: {user_input}
    Intent:
    """

    response = llm.invoke([SystemMessage(content=intent_prompt)])
    intent = response.content.strip().lower()

    return intent

# ✅ 6️⃣ Kullanıcı ile Konuşma Döngüsü
def agent_conversation():
    """
    - First, determine the user's intent.
    - If unclear, ask for clarification.
    - Once intent is clear, execute the relevant tool.
    """
    user_message = input("Please enter a banking operation: ")

    while True:
        # ✅ 1️⃣ Öncelikle Kullanıcının Niyetini Belirle
        intent = detect_intent(user_message)

        # ✅ 2️⃣ Eğer Kullanıcı Selam Verdiyse, Yanıt Ver
        if intent == "greeting":
            print("Hello! How can I assist you with your banking needs?")
            user_message = input("→ Your response: ")
            continue

        # ✅ 3️⃣ Eğer Kullanıcının Niyeti Belirsizse, Açıklama İste
        elif intent == "unknown":
            print("I didn't understand your request. Could you please clarify?")
            user_message = input("→ Your response: ")
            continue

        # ✅ 4️⃣ Niyet Belli Olduğunda, Agent’i Çalıştır ve Eksik Bilgileri Tamamla
        response = agent.invoke({"input": user_message})
        output_text = response["output"]
        print(output_text)

        # ✅ 5️⃣ Eğer Eksik Bilgi Varsa, Kullanıcıdan Yanıt Al ve Devam Et
        if any(keyword in output_text.lower() for keyword in ["who", "how much", "new address"]):
            user_message = input("→ Your response: ")
            continue

        break  # İşlem tamamlandığında döngüyü bitir

# ✅ 7️⃣ Test Et!
agent_conversation()


Please enter a banking operation:  hello


Hello! How can I assist you with your banking needs?


→ Your response:  I want to pay my credi card




> Entering new AgentExecutor chain...
Sure! How much would you like to pay towards your credit card bill?

> Finished chain.
Sure! How much would you like to pay towards your credit card bill?


→ Your response:  1500 USD pay tawords credit card bill




> Entering new AgentExecutor chain...

Invoking: `credit_card_payment` with `{'amount': 1500}`


Processing credit card payment of 1500.0 TL...
Credit card payment of 1500.0 TL completed successfully.The credit card bill of 1500 USD has been successfully paid. If you need any further assistance, feel free to ask.

> Finished chain.
The credit card bill of 1500 USD has been successfully paid. If you need any further assistance, feel free to ask.
